<a href="https://colab.research.google.com/github/YaninaK/car-insurance-renewal/blob/main/notebooks/02_Car_insurance_renewal_AutoML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Прогноз пролонгации полиса автострахования

## Benchmark: AutoML (PyCaret)

## [EDA и генерация новых признаков](https://github.com/YaninaK/car-insurance-renewal/blob/main/notebooks/01_Car_insurance_renewal_EDA.ipynb)

In [1]:
!pip install pycaret -q
!pip3 install jinja2==3.0.0 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.4/484.4 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.4/153.4 kB 14.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 29.4 MB/s eta 0:

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import category_encoders as ce
from sklearn.pipeline import Pipeline

from pycaret.classification import ClassificationExperiment

In [4]:
PATH = '/content/drive/MyDrive/ML_projects/04_Car_insurance_renewal/data/01_raw/'
RANDOM_SEED = 25

## 1. Чтение данных

In [5]:
data_file = 'Задание (пролонгация).xlsx'
field_description = pd.read_excel(PATH + data_file, 0, header=None)
data = pd.read_excel(PATH + data_file, 1)

In [6]:
TARGET_NAME = 'target'

## 2. Разделение на обучающую и тестовую выбоки

In [7]:
train = data[data[TARGET_NAME] != -1]
train.shape

(7598, 30)

## 3. Подготовка данных

In [8]:
categorical_variables = data.dtypes[data.dtypes=='object'].index.tolist()
train[categorical_variables].describe().T['unique']

Filial              22
Type_ts              2
passport_region     85
VIP                  2
Brand               23
Model              190
GAP                  2
married              2
deduct               3
deduct_amount        3
product              4
Bank                 3
channel - map        2
category             4
kvs_type             2
sex                  3
Name: unique, dtype: object

In [9]:
cols_1 = ['Filial', 'passport_region', 'Brand', 'Model']
cols_2 = [i for i in categorical_variables if not i in cols_1]

categorical_transformer = Pipeline(
    steps=[
        ("glmm_encoder", ce.glmm.GLMMEncoder(cols=cols_1,)),
        ("one_hot_encoder", ce.OneHotEncoder(cols=cols_2)),
    ]
)

In [10]:
numeric_variables = [
    i for i in train.columns[1:-1] if not i in categorical_variables
]
df = categorical_transformer.fit_transform(
    train[categorical_variables], train[TARGET_NAME]
)
X_train = pd.concat([df, train[numeric_variables + [TARGET_NAME]]], axis=1)

In [11]:
X_train.shape

(7598, 50)

## 4. Модели

In [12]:
s = ClassificationExperiment()
s.setup(X_train, target = TARGET_NAME, session_id = 123)

,Description,Value
0,Session id,123
1,Target,target
2,Target type,Binary
3,Original data shape,"(7598, 50)"
4,Transformed data shape,"(7598, 50)"
5,Transformed train set shape,"(5318, 50)"
6,Transformed test set shape,"(2280, 50)"
7,Numeric features,49
8,Preprocess,True
9,Imputation type,simple


In [13]:
best = s.compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.7401,0.7766,0.3813,0.6440,0.4774,0.3195,0.3398,1.2290
ada,Ada Boost Classifier,0.7379,0.7633,0.4139,0.6207,0.4944,0.3278,0.3409,0.5560
ridge,Ridge Classifier,0.7366,0.0000,0.3717,0.6346,0.4666,0.3080,0.3283,0.0690
lda,Linear Discriminant Analysis,0.7366,0.7582,0.3970,0.6234,0.4830,0.3186,0.3340,0.1260
lightgbm,Light Gradient Boosting Machine,0.7277,0.7635,0.4084,0.5926,0.4827,0.3067,0.3168,1.5870
rf,Random Forest Classifier,0.7245,0.7535,0.3910,0.5890,0.4687,0.2934,0.3051,1.2920
xgboost,Extreme Gradient Boosting,0.7240,0.7496,0.4512,0.5735,0.5046,0.3172,0.3218,1.5630
et,Extra Trees Classifier,0.7211,0.7387,0.4217,0.5713,0.4844,0.2997,0.3064,1.3230
nb,Naive Bayes,0.6935,0.6120,0.0699,0.5896,0.1234,0.0609,0.1139,0.0810
dummy,Dummy Classifier,0.6879,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000,0.1680


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

* Лучший AUC дает Gradient Boosting Classifier (0.7766)
* При необходимости максимизировать Recall, можно использовать Quadratic Discriminant Analysis (0.8620)